In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline

In [2]:
from cobra.io import read_sbml_model
#make sure model name is up to date!
model = read_sbml_model('iCW773_psylocibin.xml')

No objective coefficients in model. Unclear what should be optimized


In [4]:
#to set optimization (change to psylocibin)
with model:
    model.objective = model.reactions.EX_ac_e
    print(model.optimize().objective_value)

5.194057110274895


In [3]:
print(model.objective)

Maximize
0


In [5]:
import escher
model.optimize().objective_value

5.194057110274895


In [6]:
from cobra.io import read_sbml_model
model = read_sbml_model('iCW773_test7.xml')

No objective coefficients in model. Unclear what should be optimized


In [ ]:
from IPython.display import display 
from escher import Builder

In [ ]:
escher.list_available_maps()

In [ ]:
escher.Builder('e_coli_core.Core metabolism').save_html('PsyloCybinPathway.html')